# Universal Overture Feature Retriever Example

This notebook demonstrates the new universal feature retriever infrastructure that extends the BuildingRetriever pattern to support any Overture Maps feature type.

## Features
- **OvertureFeatureRetriever**: Query any single Overture feature type
- **MultiFeatureRetriever**: Query multiple feature types simultaneously
- **Theme/Type Discovery**: Automatically discover available data
- **Schema Introspection**: Get schema information for any feature type

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add functions directory to path
sys.path.append('functions')

from overture_feature_retriever import (
    OvertureFeatureRetriever,
    MultiFeatureRetriever,
    FeatureTypeInfo
)

import pandas as pd
print("✅ Imports successful")

## 1. Discover Available Data

First, let's discover what Overture themes and feature types are available in our data directory.

In [ ]:
# Set your Overture data path
# Adjust this path to point to your Overture data directory
data_path = Path("/workspace/gis_data")  # or wherever your Overture data is stored

# Discover available themes
themes = OvertureFeatureRetriever.get_available_themes(data_path)
print(f"Available themes: {themes}")

# Discover feature types for each theme
for theme in themes:
    types = OvertureFeatureRetriever.get_available_types(data_path, theme)
    print(f"  {theme}: {types}")

## 2. Single Feature Type Analysis

Use `OvertureFeatureRetriever` to analyze a single feature type.

In [ ]:
# Initialize retriever for buildings (if available)
if 'buildings' in themes:
    building_types = OvertureFeatureRetriever.get_available_types(data_path, 'buildings')
    if 'building' in building_types:
        print("🏢 Initializing building retriever...")
        building_retriever = OvertureFeatureRetriever(data_path, 'buildings', 'building')
        
        print(f"Geometry column: {building_retriever.geometry_column}")
        print(f"File count: {building_retriever.file_count}")
        print(f"Schema info: {building_retriever.schema_info}")
    else:
        print("⚠️ Building feature type not found")
else:
    print("⚠️ Buildings theme not found")

In [ ]:
# Query a small area (adjust coordinates as needed)
if 'building_retriever' in locals():
    print("🔍 Querying small area...")
    
    # Query a 0.01 degree box (roughly 1km at equator)
    result = building_retriever.query_bbox(
        minx=-0.005, miny=-0.005, 
        maxx=0.005, maxy=0.005,
        limit=10
    )
    
    print(f"Found {len(result)} buildings")
    if not result.empty:
        print(f"Columns: {list(result.columns)}")
        print("\nFirst few rows:")
        display(result.head())
    
    # Clean up
    building_retriever.close()

## 3. Multi-Feature Type Analysis

Use `MultiFeatureRetriever` to query multiple feature types simultaneously.

In [ ]:
# Initialize multi-feature retriever
print("🔧 Initializing multi-feature retriever...")
multi_retriever = MultiFeatureRetriever(data_path)

# Add available feature types
added_types = []
for theme in themes[:2]:  # Limit to first 2 themes for demo
    types = OvertureFeatureRetriever.get_available_types(data_path, theme)
    for feature_type in types[:1]:  # Limit to first type per theme
        try:
            multi_retriever.add_feature_type(theme, feature_type)
            added_types.append((theme, feature_type))
            print(f"✅ Added {theme}/{feature_type}")
        except Exception as e:
            print(f"⚠️ Could not add {theme}/{feature_type}: {e}")

print(f"\nRegistered feature types: {multi_retriever.get_feature_types()}")

In [ ]:
# Query all registered feature types
if added_types:
    print("🔍 Querying all feature types...")
    
    results = multi_retriever.query_all_types(
        minx=-0.005, miny=-0.005,
        maxx=0.005, maxy=0.005,
        limit=5
    )
    
    print(f"\nResults summary:")
    for (theme, feature_type), df in results.items():
        print(f"  {theme}/{feature_type}: {len(df)} features")
        if not df.empty:
            print(f"    Columns: {list(df.columns)}")
    
    # Show first result in detail
    if results:
        first_key = list(results.keys())[0]
        first_result = results[first_key]
        if not first_result.empty:
            print(f"\nFirst few rows from {first_key[0]}/{first_key[1]}:")
            display(first_result.head())

# Clean up
multi_retriever.close()

## 4. Schema Introspection

Explore the schema information for different feature types.

In [ ]:
# Create a new multi-retriever for schema exploration
schema_retriever = MultiFeatureRetriever(data_path)

# Add a few feature types
for theme in themes[:2]:
    types = OvertureFeatureRetriever.get_available_types(data_path, theme)
    for feature_type in types[:1]:
        try:
            schema_retriever.add_feature_type(theme, feature_type)
        except:
            pass

# Get schema information for all registered types
all_schemas = schema_retriever.get_all_schema_info()

for (theme, feature_type), schema in all_schemas.items():
    print(f"\n📋 Schema for {theme}/{feature_type}:")
    print(f"  Geometry column: {schema.get('geometry_column')}")
    print(f"  File count: {schema.get('file_count')}")
    print(f"  Columns ({len(schema.get('columns', {}))}):"))
    
    columns = schema.get('columns', {})
    for col_name, col_type in list(columns.items())[:10]:  # Show first 10 columns
        print(f"    {col_name}: {col_type}")
    
    if len(columns) > 10:
        print(f"    ... and {len(columns) - 10} more columns")

schema_retriever.close()

## Summary

The universal feature retriever infrastructure provides:

1. **Unified API**: Same interface for all Overture feature types
2. **Discovery**: Automatic detection of available themes and types
3. **Schema Introspection**: Dynamic schema information
4. **Multi-type Support**: Query multiple feature types simultaneously
5. **Performance**: Reuses BuildingRetriever's optimized spatial indexing

This foundation enables the next tasks in the implementation plan:
- Source analysis and extraction
- Spatial aggregation and sampling
- Interactive visualizations
- Comparative analysis across feature types